## 📈 Predicting Final League Positions Using Betting Odds, Probabilistic Modelling & Simulation

**Competition:** Premier League, Serie A, La Liga, Bundesliga, Ligue 1 (Season 2025/26)  
**Purpose:** Estimate probabilities of final league positions using betting market information and simulation  
**Methods:** Odds-implied probabilities, Monte Carlo simulation, scenario analysis  
**Author:** [Victoria Friss de Kereki](https://www.linkedin.com/in/victoria-friss-de-kereki/)  
**Medium Articles:**  TBC  

---

**Notebook first written:** `07/02/2026`  
**Last updated:** `08/02/2026`  

> This notebook develops a probabilistic framework to predict final league positions across Europe’s top five leagues (England, Italy, Spain, Germany, France) using betting odds as market-based expectations.
>
> Betting odds are transformed into implied probabilities and adjusted for bookmaker margins. These probabilities are then used to simulate the remainder of each season via Monte Carlo methods, generating distributions over final points totals and league positions.
>
> The analysis estimates the likelihood of key outcomes such as league titles, European qualification, relegation, and mid-table finishes. Results are presented at the team level with uncertainty intervals. The framework can be extended to incorporate additional inputs such as recent form, fixture difficulty, or alternative predictive models beyond betting markets.


<div style="text-align: left;">
    <img src="Images and others/Predicting Premier League Final Positions Using Betting Odds, Probabilistic Modelling & Simulation.png" alt="Predicting Premier League Final Positions Using Betting Odds, Probabilistic Modelling & Simulation" width="600">
</di>
>

In [1]:
# Core
from datetime import datetime, timedelta
import os

# Data manipulation
import numpy as np
import pandas as pd

# APIs & environment
import requests
from dotenv import load_dotenv
import time  # for delaying requests

# Statistics
from scipy.stats import poisson

# Visualisation
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Nicer printing of tables, no wrapping
pd.set_option("display.width", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

## 1. Final Standings (ESPN Scraping) for the 5 leagues

In [2]:
import pandas as pd

year = 2025  # current season start year

leagues = {
    "ENG.1": "premierleague_england",
    "ITA.1": "seriea_italy",
    "ESP.1": "laliga_spain",
    "GER.1": "bundesliga_germany",
    "FRA.1": "ligue1_france",
}

for league_code, df_name in leagues.items():
    url = f"https://www.espn.com/soccer/standings/_/league/{league_code}/season/{year}"
    tables = pd.read_html(url)

    teams_raw = tables[0]
    stats = tables[1]

    teams = pd.DataFrame()
    teams["position"] = teams_raw.iloc[:, 0].str.extract(r"^(\d+)").astype(int)
    teams["team"] = (
        teams_raw.iloc[:, 0]
        .str.replace(r"^\d+", "", regex=True)
        .str.replace(r"^[A-Z]{2,3}", "", regex=True)
        .str.strip()
    )

    stats.columns = ["gp", "w", "d", "l", "gf", "ga", "gd", "pts"]
    stats = stats.apply(
        lambda c: c.astype(str)
                  .str.replace("+", "", regex=False)
                  .astype(int)
    )

    globals()[df_name] = pd.concat([teams, stats], axis=1)

In [3]:
print("\nPremier League (England)")
print(premierleague_england.head(3))

print("\nSerie A (Italy)")
print(seriea_italy.head(3))

print("\nLa Liga (Spain)")
print(laliga_spain.head(3))

print("\nBundesliga (Germany)")
print(bundesliga_germany.head(3))

print("\nLigue 1 (France)")
print(ligue1_france.head(3))


Premier League (England)
   position             team  gp   w  d  l  gf  ga  gd  pts
0         1          Arsenal  25  17  5  3  49  17  32   56
1         2  Manchester City  24  14  5  5  49  23  26   47
2         3      Aston Villa  25  14  5  6  36  27   9   47

Serie A (Italy)
   position            team  gp   w  d  l  gf  ga  gd  pts
0         1  Internazionale  23  18  1  4  52  19  33   55
1         2        AC Milan  23  14  8  1  38  17  21   50
2         3          Napoli  24  15  4  5  36  23  13   49

La Liga (Spain)
   position             team  gp   w  d  l  gf  ga  gd  pts
0         1        Barcelona  23  19  1  3  63  23  40   58
1         2      Real Madrid  22  17  3  2  47  18  29   54
2         3  Atlético Madrid  22  13  6  3  38  17  21   45

Bundesliga (Germany)
   position               team  gp   w  d  l  gf  ga  gd  pts
0         1      Bayern Munich  20  16  3  1  74  18  56   51
1         2  Borussia Dortmund  21  14  6  1  43  20  23   48
2         3     

In [4]:
leagues_data = {
    "Premier League (England)": premierleague_england,
    "Serie A (Italy)": seriea_italy,
    "La Liga (Spain)": laliga_spain,
    "Bundesliga (Germany)": bundesliga_germany,
    "Ligue 1 (France)": ligue1_france,
}

matches_unplayed_ = {}

for league_name, df in leagues_data.items():
    num_teams = len(df)
    total_matches = num_teams * (num_teams - 1)  # double round-robin, total matches counted twice for GP
    matches_played = df["gp"].sum() /2              # GP already counts each match per team
    matches_unplayed = total_matches - matches_played
    
    matches_unplayed_[league_name] = matches_unplayed
    print(f"{league_name}: {matches_unplayed} matches unplayed")

Premier League (England): 131.0 matches unplayed
Serie A (Italy): 145.0 matches unplayed
La Liga (Spain): 155.0 matches unplayed
Bundesliga (Germany): 119.0 matches unplayed
Ligue 1 (France): 121.0 matches unplayed


## 2. Get betting odds using API

In [5]:
# Load variables from API_KEY.env
load_dotenv("API_KEY.env")

API_KEY = os.getenv("ODDS_DATA_API_KEY")

if API_KEY is None:
    raise ValueError("API_KEY not found. Check API_KEY.env")

print("API key loaded successfully")

API key loaded successfully


In [6]:
import requests

API_KEY = API_KEY  # assuming already defined

leagues = {
    "soccer_epl": "odds_premierleague_england",
    "soccer_italy_serie_a": "odds_seriea_italy",
    "soccer_spain_la_liga": "odds_laliga_spain",
    "soccer_germany_bundesliga": "odds_bundesliga_germany",
    "soccer_france_ligue_one": "odds_ligue1_france",
}

base_url = "https://api.the-odds-api.com/v4/sports/{}/odds"

params = {
    "apiKey": API_KEY,
    "regions": "uk",
    "markets": "h2h",
    "oddsFormat": "decimal",
    "dateFormat": "iso",
    "days": 365
}

for sport_key, var_name in leagues.items():
    url = base_url.format(sport_key)

    response = requests.get(url, params=params)
    response.raise_for_status()

    globals()[var_name] = response.json()

In [7]:
print("Premier League (England):", len(odds_premierleague_england))
print("Serie A (Italy):", len(odds_seriea_italy))
print("La Liga (Spain):", len(odds_laliga_spain))
print("Bundesliga (Germany):", len(odds_bundesliga_germany))
print("Ligue 1 (France):", len(odds_ligue1_france))

Premier League (England): 22
Serie A (Italy): 18
La Liga (Spain): 24
Bundesliga (Germany): 19
Ligue 1 (France): 22


In [8]:
def flatten_odds(data):
    rows = []

    for match in data:
        match_id = match["id"]
        home = match["home_team"]
        away = match["away_team"]
        time = match["commence_time"]

        for book in match["bookmakers"]:
            bookmaker = book["title"]

            # Find head-to-head (h2h) market. Find the market where key == 'h2h' (win/draw/win odds). If not found, skip this bookmaker.
            h2h = next((m for m in book["markets"] if m["key"] == "h2h"), None)
            if not h2h:
                continue

            outcomes = {o["name"]: o["price"] for o in h2h["outcomes"]}

            rows.append({
                "match_id": match_id,
                "commence_time": time,
                "home_team": home,
                "away_team": away,
                "bookmaker": bookmaker,
                "home_odds": outcomes.get(home),
                "draw_odds": outcomes.get("Draw"),
                "away_odds": outcomes.get(away),
            })

    return pd.DataFrame(rows)

In [9]:
# Flatten odds into DataFrames
df_premierleague_england = flatten_odds(odds_premierleague_england)
df_seriea_italy = flatten_odds(odds_seriea_italy)
df_laliga_spain = flatten_odds(odds_laliga_spain)
df_bundesliga_germany = flatten_odds(odds_bundesliga_germany)
df_ligue1_france = flatten_odds(odds_ligue1_france)

In [10]:
print("\nPremier League (England)")
print(df_premierleague_england.head(3))

print("\nSerie A (Italy)")
print(df_seriea_italy.head(3))

print("\nLa Liga (Spain)")
print(df_laliga_spain.head(3))

print("\nBundesliga (Germany)")
print(df_bundesliga_germany.head(3))

print("\nLigue 1 (France)")
print(df_ligue1_france.head(3))


Premier League (England)
                           match_id         commence_time  home_team        away_team    bookmaker  home_odds  draw_odds  away_odds
0  e526c157d950230afb02bc6534fbaf82  2026-02-08T16:30:00Z  Liverpool  Manchester City  Unibet (UK)       3.10       2.15        3.6
1  e526c157d950230afb02bc6534fbaf82  2026-02-08T16:30:00Z  Liverpool  Manchester City  Paddy Power       3.25       2.00        3.5
2  e526c157d950230afb02bc6534fbaf82  2026-02-08T16:30:00Z  Liverpool  Manchester City      Sky Bet       3.30       2.10        3.5

Serie A (Italy)
                           match_id         commence_time home_team away_team     bookmaker  home_odds  draw_odds  away_odds
0  b8471d4a11ea8dd2f1906a0cc4739ed2  2026-02-08T19:45:00Z  Juventus     Lazio        Betway        1.4        4.5        8.0
1  b8471d4a11ea8dd2f1906a0cc4739ed2  2026-02-08T19:45:00Z  Juventus     Lazio  William Hill        1.4        4.0        8.0
2  b8471d4a11ea8dd2f1906a0cc4739ed2  2026-02-08T19:45:

In [11]:
def bookmaker_implied_probs(df):
    # Convert odds to implied probabilities per bookmaker
    df = df.assign(
        p_home_raw=1 / df["home_odds"],
        p_draw_raw=1 / df["draw_odds"],
        p_away_raw=1 / df["away_odds"],
    )

    # Remove bookmaker margin (normalise)
    total = (
        df["p_home_raw"] +
        df["p_draw_raw"] +
        df["p_away_raw"]
    )

    df = df.assign(
        p_home_book=df["p_home_raw"] / total,
        p_draw_book=df["p_draw_raw"] / total,
        p_away_book=df["p_away_raw"] / total,
    )

    # Average normalised probabilities across bookmakers
    betting_odds_avg = (
        df.groupby(["match_id", "home_team", "away_team"], as_index=False)
          .agg(
              p_home_book=("p_home_book", "mean"),
              p_draw_book=("p_draw_book", "mean"),
              p_away_book=("p_away_book", "mean"),
          )
    )

    # Keep only required fields
    betting_odds_avg = betting_odds_avg[
        [
            "home_team",
            "away_team",
            "p_home_book",
            "p_draw_book",
            "p_away_book",
        ]
    ]

    return betting_odds_avg

In [12]:
betting_odds_premierleague_england = bookmaker_implied_probs(df_premierleague_england)
betting_odds_seriea_italy = bookmaker_implied_probs(df_seriea_italy)
betting_odds_laliga_spain = bookmaker_implied_probs(df_laliga_spain)
betting_odds_bundesliga_germany = bookmaker_implied_probs(df_bundesliga_germany)
betting_odds_ligue1_france = bookmaker_implied_probs(df_ligue1_france)

In [13]:
print("\nPremier League (England)")
print(betting_odds_premierleague_england.head(3))

print("\nSerie A (Italy)")
print(betting_odds_seriea_italy.head(3))

print("\nLa Liga (Spain)")
print(betting_odds_laliga_spain.head(3))

print("\nBundesliga (Germany)")
print(betting_odds_bundesliga_germany.head(3))

print("\nLigue 1 (France)")
print(betting_odds_ligue1_france.head(3))


Premier League (England)
        home_team                away_team  p_home_book  p_draw_book  p_away_book
0  Crystal Palace                  Burnley     0.597846     0.238591     0.163563
1  Crystal Palace  Wolverhampton Wanderers     0.526722     0.264405     0.208873
2     Aston Villa             Leeds United     0.497548     0.264353     0.238099

Serie A (Italy)
     home_team away_team  p_home_book  p_draw_book  p_away_book
0     Cagliari     Lazio     0.292117     0.321401     0.386482
1  Inter Milan  Juventus     0.449597     0.283866     0.266537
2         Pisa  AC Milan     0.151612     0.237166     0.611222

La Liga (Spain)
    home_team   away_team  p_home_book  p_draw_book  p_away_book
0      Getafe  Villarreal     0.321686     0.309309     0.369005
1  Celta Vigo    Mallorca     0.525009     0.275874     0.199117
2  Villarreal    Espanyol     0.562071     0.237323     0.200606

Bundesliga (Germany)
       home_team       away_team  p_home_book  p_draw_book  p_away_book
0 

## 3. Get fixtures for upcoming EPL games

In [14]:
# Load variables from API_KEY.env
load_dotenv("API_KEY.env")

API_KEY = os.getenv("FOOTBALL_DATA_API_KEY")

if API_KEY is None:
    raise ValueError("API_KEY not found. Check API_KEY.env")

print("API key loaded successfully")

API key loaded successfully


In [15]:
competitions = {
    "PL": "fixtures_premierleague_england",
    "SA": "fixtures_seriea_italy",
    "PD": "fixtures_laliga_spain",
    "BL1": "fixtures_bundesliga_germany",
    "FL1": "fixtures_ligue1_france",
}

headers = {
    "X-Auth-Token": API_KEY
}

today = datetime.utcnow().date()
end_of_season = today + timedelta(days=365)

params = {
    "status": "SCHEDULED",
    "dateFrom": today.isoformat(),
    "dateTo": end_of_season.isoformat()
}

for comp_code, df_name in competitions.items():
    url = f"https://api.football-data.org/v4/competitions/{comp_code}/matches"

    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()

    data = response.json()
    fixtures = data["matches"]

    df_fixtures = pd.DataFrame(fixtures)

    df_fixtures_clean = df_fixtures[
        ["utcDate", "status", "homeTeam", "awayTeam"]
    ].copy()  # copy avoids SettingWithCopyWarning

    # Extract team names
    df_fixtures_clean["homeTeam"] = df_fixtures_clean["homeTeam"].apply(lambda x: x["name"])
    df_fixtures_clean["awayTeam"] = df_fixtures_clean["awayTeam"].apply(lambda x: x["name"])

    globals()[df_name] = df_fixtures_clean

C:\Users\vicky\AppData\Local\Temp\ipykernel_18804\3061059445.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = datetime.utcnow().date()


In [16]:
print("Premier League (England):", len(fixtures_premierleague_england))
print("Serie A (Italy):", len(fixtures_seriea_italy))
print("La Liga (Spain):", len(fixtures_laliga_spain))
print("Bundesliga (Germany):", len(fixtures_bundesliga_germany))
print("Ligue 1 (France):", len(fixtures_ligue1_france))

Premier League (England): 129
Serie A (Italy): 144
La Liga (Spain): 153
Bundesliga (Germany): 118
Ligue 1 (France): 118


In [17]:
print("Premier League (England):", fixtures_premierleague_england.head(3))
print("Serie A (Italy):", fixtures_seriea_italy.head(3))
print("La Liga (Spain):", fixtures_laliga_spain.head(3))
print("Bundesliga (Germany):", fixtures_bundesliga_germany.head(3))
print("Ligue 1 (France):", fixtures_ligue1_france.head(3))

Premier League (England):                 utcDate status              homeTeam             awayTeam
0  2026-02-10T19:30:00Z  TIMED            Chelsea FC      Leeds United FC
1  2026-02-10T19:30:00Z  TIMED            Everton FC      AFC Bournemouth
2  2026-02-10T19:30:00Z  TIMED  Tottenham Hotspur FC  Newcastle United FC
Serie A (Italy):                 utcDate status     homeTeam         awayTeam
0  2026-02-08T19:45:00Z  TIMED  Juventus FC         SS Lazio
1  2026-02-09T17:30:00Z  TIMED  Atalanta BC     US Cremonese
2  2026-02-09T19:45:00Z  TIMED      AS Roma  Cagliari Calcio
La Liga (Spain):                 utcDate status       homeTeam                   awayTeam
0  2026-02-08T20:00:00Z  TIMED    Valencia CF             Real Madrid CF
1  2026-02-09T20:00:00Z  TIMED  Villarreal CF  RCD Espanyol de Barcelona
2  2026-02-13T20:00:00Z  TIMED       Elche CF                 CA Osasuna
Bundesliga (Germany):                 utcDate status             homeTeam           awayTeam
0  2026-02-13T1

## 4. Get this season (2025/26) and last season (2024/25) results

In [18]:
competitions = {
    "PL": "premierleague_england",
    "SA": "seriea_italy",
    "PD": "laliga_spain",
    "BL1": "bundesliga_germany",
    "FL1": "ligue1_france",
}

seasons = [2025, 2024]  # finished seasons you want
headers = {
    "X-Auth-Token": API_KEY
}

for comp_code, league_name in competitions.items():
    for season in seasons:
        print(f"Fetching {league_name} season {season}...")
        url = f"https://api.football-data.org/v4/competitions/{comp_code}/matches"
        params = {
            "season": season,
            "status": "FINISHED"
        }

        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        matches = response.json()["matches"]

        clean_rows = []
        for m in matches:
            clean_rows.append({
                "utcDate": m["utcDate"],
                "matchday": m["matchday"],
                "status": m["status"],
                "homeTeam": m["homeTeam"]["name"],
                "awayTeam": m["awayTeam"]["name"],
                "homeGoals": m["score"]["fullTime"]["home"],
                "awayGoals": m["score"]["fullTime"]["away"],
                "winner": m["score"]["winner"],
            })

        df_clean = pd.DataFrame(clean_rows)

        globals()[f"past_matches_{league_name}_{season}_clean"] = df_clean
        
        # Wait a few seconds between each API request to avoid crushing
        time.sleep(10)

print("Done fetching all matches.")

Fetching premierleague_england season 2025...
Fetching premierleague_england season 2024...
Fetching seriea_italy season 2025...
Fetching seriea_italy season 2024...
Fetching laliga_spain season 2025...
Fetching laliga_spain season 2024...
Fetching bundesliga_germany season 2025...
Fetching bundesliga_germany season 2024...
Fetching ligue1_france season 2025...
Fetching ligue1_france season 2024...
Done fetching all matches.


In [19]:
for league in [
    "premierleague_england",
    "seriea_italy",
    "laliga_spain",
    "bundesliga_germany",
    "ligue1_france",
]:
    for season in [2025, 2024]:
        df = globals()[f"past_matches_{league}_{season}_clean"]
        print(f"\n{league.replace('_', ' ').title()} – Season {season}")
        print(df.tail(2))


Premierleague England – Season 2025
                  utcDate  matchday    status                   homeTeam           awayTeam  homeGoals  awayGoals     winner
247  2026-02-07T17:30:00Z        25  FINISHED        Newcastle United FC       Brentford FC          2          3  AWAY_TEAM
248  2026-02-08T14:00:00Z        25  FINISHED  Brighton & Hove Albion FC  Crystal Palace FC          0          1  AWAY_TEAM

Premierleague England – Season 2024
                  utcDate  matchday    status                    homeTeam                   awayTeam  homeGoals  awayGoals     winner
378  2025-05-25T15:00:00Z        38  FINISHED        Tottenham Hotspur FC  Brighton & Hove Albion FC          1          4  AWAY_TEAM
379  2025-05-25T15:00:00Z        38  FINISHED  Wolverhampton Wanderers FC               Brentford FC          1          1       DRAW

Seriea Italy – Season 2025
                  utcDate  matchday    status         homeTeam           awayTeam  homeGoals  awayGoals     winner
233  2

In [20]:
# Combine datasets into past_matches_ and future_matches_

leagues = [
    "premierleague_england",
    "seriea_italy",
    "laliga_spain",
    "bundesliga_germany",
    "ligue1_france",
]

for league in leagues:
    # Load DataFrames
    df_current = globals()[f"past_matches_{league}_2025_clean"]
    df_prev = globals()[f"past_matches_{league}_2024_clean"]
    df_future = globals()[f"fixtures_{league}"]

    # Combine all past fixtures together
    df_all = pd.concat([df_prev, df_current], ignore_index=True)

    # Store results
    globals()[f"past_matches_{league}_all"] = df_all
    globals()[f"future_matches_{league}"] = df_future

## 5. Unify team names across datasets

In [21]:
# === Premier League ===
mapping_premierleague_england = {
    # df_probabilities
    "Aston Villa FC": "Aston Villa",
    "Leeds United FC": "Leeds United",
    "Newcastle United FC": "Newcastle United",
    "Crystal Palace FC": "Crystal Palace",
    "Chelsea FC": "Chelsea",
    "Arsenal FC": "Arsenal",
    "Everton FC": "Everton",
    "Burnley FC": "Burnley",
    "Brighton & Hove Albion FC": "Brighton & Hove Albion",
    "Sunderland AFC": "Sunderland",
    "West Ham United FC": "West Ham United",
    "Manchester City FC": "Manchester City",
    "Manchester United FC": "Manchester United",
    "Fulham FC": "Fulham",
    "Liverpool FC": "Liverpool",
    "Brentford FC": "Brentford",
    "Wolverhampton Wanderers FC": "Wolverhampton Wanderers",
    "Nottingham Forest FC": "Nottingham Forest",
    "Tottenham Hotspur FC": "Tottenham Hotspur",
    # betting_odds
    "Brighton and Hove Albion": "Brighton & Hove Albion",
    "Bournemouth": "AFC Bournemouth"
}

# === Serie A ===
mapping_seriea_italy = {
    # df_probabilities
    "US Sassuolo Calcio": "Sassuolo",
    "Cagliari Calcio": "Cagliari",
    "Atalanta BC": "Atalanta",
    "SS Lazio": "Lazio",
    "Genoa CFC": "Genoa",
    "Udinese Calcio": "Udinese",
    "FC Internazionale Milano": "Internazionale",
    "Torino FC": "Torino",
    "AC Pisa 1909": "Pisa",
    "ACF Fiorentina": "Fiorentina",
    "AS Roma": "AAS Roma",
    "Juventus FC": "Juventus",
    "Como 1907": "OComo",
    "US Cremonese": "Cremonese",
    "Bologna FC 1909": "Bologna",
    "Parma Calcio 1913": "Parma",
    "Hellas Verona FC": "Hellas Verona",
    "SSC Napoli": "Napoli",
    "US Lecce": "Lecce",
    # betting_odds
    "Inter Milan": "Internazionale",
    "Como": "OComo"
}

# === La Liga ===
mapping_laliga_spain = {
    # df_probabilities
    "Club Atlético de Madrid": "Atlético Madrid",
    "Rayo Vallecano de Madrid": "Rayo Vallecano",
    "Valencia CF": "Valencia",
    "Deportivo Alavés": "Alavés",
    "CA Osasuna": "Osasuna",
    "RCD Espanyol de Barcelona": "Espanyol",
    "Getafe CF": "Getafe",
    "Real Sociedad de Fútbol": "Real Sociedad",
    "Levante UD": "Levante",
    "Real Betis Balompié": "Real Betis",
    "RCD Mallorca": "Mallorca",
    "Girona FC": "Girona",
    "Villarreal CF": "Villarreal",
    "FC Barcelona": "Barcelona",
    "Elche CF": "Elche",
    "Sevilla FC": "Sevilla",
    "Real Madrid CF": "Real Madrid",
    "RC Celta de Vigo": "Celta Vigo",
    # betting_odds
    "Elche CF": "Elche",
    "Oviedo": "Real Oviedo",
    "CA Osasuna": "Osasuna",
    "Athletic Bilbao": "Athletic Club"
}

# === Bundesliga ===
mapping_bundesliga_germany = {
    # df_probabilities
    "1. FC Köln": "FC Cologne",
    "TSG 1899 Hoffenheim": "TSG Hoffenheim",
    "1. FSV Mainz 05": "M05Mainz",
    "SV Werder Bremen": "Werder Bremen",
    "Hamburger SV": "Hamburg SV",
    "Bayer 04 Leverkusen": "B04Bayer Leverkusen",
    "FC St. Pauli 1910": "St. Pauli",
    "FC Bayern München": "Bayern Munich",
    # betting_odds
    "1. FC Heidenheim": "1. FC Heidenheim 1846",
    "Union Berlin": "1. FC Union Berlin",
    "Borussia Monchengladbach": "Borussia Mönchengladbach",
    "FSV Mainz 05": "M05Mainz",
    "Bayer Leverkusen": "B04Bayer Leverkusen",
    "Augsburg": "FC Augsburg",
    "FC St. Pauli": "St. Pauli"
}

# === Ligue 1 ===
mapping_ligue1_france = {
    # df_probabilities
    "Racing Club de Lens": "Lens",
    "OGC Nice": "ENice",
    "FC Metz": "ZMetz",
    "Angers SCO": "Angers",
    "Stade Brestois 29": "Brest",
    "Olympique Lyonnais": "NLyon",
    "Paris Saint-Germain FC": "Paris Saint-Germain",
    "AS Monaco FC": "AS Monaco",
    "Lille OSC": "LLille",
    "Toulouse FC": "Toulouse",
    "FC Nantes": "Nantes",
    "RC Strasbourg Alsace": "Strasbourg",
    "FC Lorient": "Lorient",
    "Olympique de Marseille": "Marseille",
    "Stade Rennais FC 1901": "Stade Rennais",
    # betting_odds
    "RC Lens": "Lens",
    "Paris Saint Germain": "Paris Saint-Germain",
    "Auxerre": "AJ Auxerre",
    "Lyon": "NLyon",
    "Le Havre": "Le Havre AC",
    "Rennes": "Stade Rennais",
    "Metz": "ZMetz",
    "Nice": "ENice",
    "Lille": "LLille"
}

In [22]:
# Mapping dictionaries per league
mappings = {
    "premierleague_england": mapping_premierleague_england,
    "seriea_italy": mapping_seriea_italy,
    "laliga_spain": mapping_laliga_spain,
    "bundesliga_germany": mapping_bundesliga_germany,
    "ligue1_france": mapping_ligue1_france,
}

# Datasets to unify
datasets_templates = [
    "past_matches_{}_all",
    "future_matches_{}",
    "betting_odds_{}"
]

for league, mapping in mappings.items():
    print(f"Applying mapping for {league}...")
    
    for ds_template in datasets_templates:
        ds_name = ds_template.format(league)
        df = globals()[ds_name]
        
        # Replace team names using the mapping
        df.replace(mapping, inplace=True)

        print(f"  Updated {ds_name} team names.")

Applying mapping for premierleague_england...
  Updated past_matches_premierleague_england_all team names.
  Updated future_matches_premierleague_england team names.
  Updated betting_odds_premierleague_england team names.
Applying mapping for seriea_italy...
  Updated past_matches_seriea_italy_all team names.
  Updated future_matches_seriea_italy team names.
  Updated betting_odds_seriea_italy team names.
Applying mapping for laliga_spain...
  Updated past_matches_laliga_spain_all team names.
  Updated future_matches_laliga_spain team names.
  Updated betting_odds_laliga_spain team names.
Applying mapping for bundesliga_germany...
  Updated past_matches_bundesliga_germany_all team names.
  Updated future_matches_bundesliga_germany team names.
  Updated betting_odds_bundesliga_germany team names.
Applying mapping for ligue1_france...
  Updated past_matches_ligue1_france_all team names.
  Updated future_matches_ligue1_france team names.
  Updated betting_odds_ligue1_france team names.


## 6. Check for missing fixtures (rescheduled matches)

In [23]:
for league in leagues:
    future_matches = globals()[f"future_matches_{league}"]
    betting_odds = globals()[f"betting_odds_{league}"]

    # Identify bookmaker matches missing from future_matches
    future_set = set(zip(future_matches["homeTeam"], future_matches["awayTeam"]))
    book_set = set(zip(betting_odds["home_team"], betting_odds["away_team"]))
    missing_matches = book_set - future_set

    # If any are missing, append them to the future_matches DataFrame
    if missing_matches:
        print(f"{league}: Adding {len(missing_matches)} missing matches from bookmaker to future_matches")
        rows_to_add = []
        for home_team, away_team in missing_matches:
            row = betting_odds[
                (betting_odds["home_team"] == home_team) &
                (betting_odds["away_team"] == away_team)
            ].iloc[0]  # take first row if duplicates
            rows_to_add.append({
                "utcDate": row.get("utcDate", pd.NaT),  # fallback if missing
                "homeTeam": home_team,
                "awayTeam": away_team,
            })

        future_matches = pd.concat([future_matches, pd.DataFrame(rows_to_add)], ignore_index=True)
        globals()[f"future_matches_{league}"] = future_matches
        print(f"✅ Added {len(rows_to_add)} matches to future_matches")
    else:
        print(f"{league}: No missing matches from bookmaker odds")

premierleague_england: Adding 1 missing matches from bookmaker to future_matches
✅ Added 1 matches to future_matches
seriea_italy: No missing matches from bookmaker odds
laliga_spain: Adding 1 missing matches from bookmaker to future_matches
✅ Added 1 matches to future_matches
bundesliga_germany: Adding 1 missing matches from bookmaker to future_matches
✅ Added 1 matches to future_matches
ligue1_france: Adding 3 missing matches from bookmaker to future_matches
✅ Added 3 matches to future_matches


In [24]:
leagues = [
    "premierleague_england",
    "seriea_italy",
    "laliga_spain",
    "bundesliga_germany",
    "ligue1_france",
]

df_compare_all = {}

for league in leagues:
    print(f"\n=== {league.replace('_',' ').title()} ===")
    
    # Load tables
    league_table = globals()[league]                   # 'team', 'gp'
    future_matches = globals()[f"future_matches_{league}"]
    betting_odds = globals()[f"betting_odds_{league}"]

    # --- Compare future matches to bookmaker rows ---
    df_compare = future_matches.merge(
        betting_odds,
        left_on=["homeTeam", "awayTeam"],
        right_on=["home_team", "away_team"],
        how="outer",
        indicator=True
    )

    total_future_matches = len(future_matches)
    total_odds_rows = len(betting_odds)
    matched_rows = len(df_compare[df_compare["_merge"] == "both"])
    missing_in_odds = len(df_compare[df_compare["_merge"] == "left_only"])
    missing_in_future = len(df_compare[df_compare["_merge"] == "right_only"])

    print(f"Total future matches: {total_future_matches}")
    print(f"Total bookmaker rows: {total_odds_rows}")
    print(f"Matched rows: {matched_rows}")
    print(f"Missing in bookmaker: {missing_in_odds}")
    print(f"Missing in future matches table: {missing_in_future}")

    # --- Count per-team missing matches ---
    total_teams = len(league_table)
    matches_per_team_season = (total_teams - 1) * 2  # double round-robin

    # Count how many future matches each team has
    future_counts = {
        team: (
            future_matches[future_matches['homeTeam'] == team].shape[0] +
            future_matches[future_matches['awayTeam'] == team].shape[0]
        )
        for team in league_table['team']
    }

    # Compare expected vs actual
    missing_matches = {}
    for idx, row in league_table.iterrows():
        team = row['team']
        expected_future = matches_per_team_season - row['gp']
        actual_future = future_counts.get(team, 0)
        if expected_future != actual_future:
            missing_matches[team] = expected_future - actual_future

    if missing_matches:
        print("⚠️ Teams missing future matches:")
        for team, missing in missing_matches.items():
            print(f"  {team}: {missing} missing")
    else:
        print("✅ All teams have the expected number of future matches")

    # Save comparison dataframe for later
    df_compare_all[league] = df_compare


=== Premierleague England ===
Total future matches: 130
Total bookmaker rows: 22
Matched rows: 22
Missing in bookmaker: 108
Missing in future matches table: 0
⚠️ Teams missing future matches:
  Manchester City: 1 missing
  Crystal Palace: 1 missing

=== Seriea Italy ===
Total future matches: 144
Total bookmaker rows: 18
Matched rows: 18
Missing in bookmaker: 126
Missing in future matches table: 0
⚠️ Teams missing future matches:
  Internazionale: 1 missing
  Sassuolo: 1 missing

=== Laliga Spain ===
Total future matches: 154
Total bookmaker rows: 24
Matched rows: 24
Missing in bookmaker: 130
Missing in future matches table: 0
⚠️ Teams missing future matches:
  Rayo Vallecano: 1 missing
  Real Oviedo: 1 missing

=== Bundesliga Germany ===
Total future matches: 119
Total bookmaker rows: 19
Matched rows: 19
Missing in bookmaker: 100
Missing in future matches table: 0
✅ All teams have the expected number of future matches

=== Ligue1 France ===
Total future matches: 121
Total bookmaker ro

## 7. Combine and calculate probabilities of W/D/L for each match

In [25]:
leagues = [
    "premierleague_england",
    "seriea_italy",
    "laliga_spain",
    "bundesliga_germany",
    "ligue1_france",
]

for league in leagues:
    df_all = globals()[f"past_matches_{league}_all"].copy()

    # Convert date
    df_all["date"] = pd.to_datetime(df_all["utcDate"])

    # Sort so newer matches get higher weight
    df_all = df_all.sort_values("date").reset_index(drop=True)

    # Add linear weights (oldest → newest)
    df_all["weight"] = np.linspace(1, 2, len(df_all))

    # Store weighted dataset
    globals()[f"past_matches_{league}_weighted"] = df_all

In [26]:
for league in [
    "premierleague_england",
    "seriea_italy",
    "laliga_spain",
    "bundesliga_germany",
    "ligue1_france",
]:
    df = globals()[f"past_matches_{league}_weighted"]
    print(f"\n{league.replace('_', ' ').title()} – weighted past matches")
    print(df.tail(2))


Premierleague England – weighted past matches
                  utcDate  matchday    status                homeTeam        awayTeam  homeGoals  awayGoals     winner                      date    weight
627  2026-02-07T17:30:00Z        25  FINISHED        Newcastle United       Brentford          2          3  AWAY_TEAM 2026-02-07 17:30:00+00:00  1.998408
628  2026-02-08T14:00:00Z        25  FINISHED  Brighton & Hove Albion  Crystal Palace          0          1  AWAY_TEAM 2026-02-08 14:00:00+00:00  2.000000

Seriea Italy – weighted past matches
                  utcDate  matchday    status homeTeam awayTeam  homeGoals  awayGoals     winner                      date    weight
613  2026-02-08T11:30:00Z        24  FINISHED  Bologna    Parma          0          1  AWAY_TEAM 2026-02-08 11:30:00+00:00  1.998371
614  2026-02-08T14:00:00Z        24  FINISHED    Lecce  Udinese          2          1  HOME_TEAM 2026-02-08 14:00:00+00:00  2.000000

Laliga Spain – weighted past matches
             

In [27]:
# compute home advantage per league and save it to globals().

leagues = [
    "premierleague_england",
    "seriea_italy",
    "laliga_spain",
    "bundesliga_germany",
    "ligue1_france",
]

home_advantage_by_league = {}

for league in leagues:
    # Access the weighted past matches for this league
    df_all = globals()[f"past_matches_{league}_weighted"]

    # Compute home advantage
    home_adv = df_all["homeGoals"].mean() - df_all["awayGoals"].mean()

    # Save to dictionary
    home_advantage_by_league[league] = home_adv

    # Save to globals (for your Poisson model)
    globals()[f"home_advantage_{league}"] = home_adv

    # Print nicely
    print(f"{league.replace('_', ' ').title()}: {home_adv:.3f}")

Premierleague England: 0.184
Seriea Italy: 0.124
Laliga Spain: 0.331
Bundesliga Germany: 0.175
Ligue1 France: 0.316


In [28]:
leagues = [
    "premierleague_england",
    "seriea_italy",
    "laliga_spain",
    "bundesliga_germany",
    "ligue1_france",
]

for league in leagues:
    df_all = globals()[f"past_matches_{league}_weighted"]

    # All teams in the league
    teams = pd.unique(df_all[["homeTeam", "awayTeam"]].values.ravel("K"))

    attack = pd.Series(1.0, index=teams)
    defense = pd.Series(1.0, index=teams)

    team_stats = {}

    for team in teams:
        home_games = df_all[df_all["homeTeam"] == team]
        away_games = df_all[df_all["awayTeam"] == team]

        goals_scored = (
            (home_games["homeGoals"] * home_games["weight"]).sum() +
            (away_games["awayGoals"] * away_games["weight"]).sum()
        )

        goals_against = (
            (home_games["awayGoals"] * home_games["weight"]).sum() +
            (away_games["homeGoals"] * away_games["weight"]).sum()
        )

        matches = home_games["weight"].sum() + away_games["weight"].sum()

        team_stats[team] = {
            "scored": goals_scored / matches,
            "against": goals_against / matches
        }

    # League average goals per team per match
    league_avg_scored = (
        df_all["homeGoals"].mean() + df_all["awayGoals"].mean()
    ) / 2

    for team in teams:
        attack[team] = team_stats[team]["scored"] / league_avg_scored
        defense[team] = team_stats[team]["against"] / league_avg_scored

    # Store outputs
    globals()[f"attack_{league}"] = attack
    globals()[f"defense_{league}"] = defense
    globals()[f"league_avg_scored_{league}"] = league_avg_scored

🔥 Summary

This function:
+ Calculates expected goals for each team
+ Uses Poisson distribution to compute goal probabilities
+ Converts score probabilities into match outcome probabilities
+ Returns probabilities for:
++ home win
++ draw
++ away win

In [29]:
def match_probabilities_league(
    home,
    away,
    attack,
    defense,
    league_avg_scored,
    home_advantage,
    max_goals=6,
):
    # expected goals
    exp_home = np.exp(
        np.log(league_avg_scored)
        + np.log(attack[home])
        + np.log(defense[away])
        + home_advantage
    )

    exp_away = np.exp(
        np.log(league_avg_scored)
        + np.log(attack[away])
        + np.log(defense[home])
    )

    p_home = poisson.pmf(range(max_goals + 1), exp_home)
    p_away = poisson.pmf(range(max_goals + 1), exp_away)

    p_win = p_draw = p_loss = 0.0

    for i in range(max_goals + 1):
        for j in range(max_goals + 1):
            prob = p_home[i] * p_away[j]
            if i > j:
                p_win += prob
            elif i == j:
                p_draw += prob
            else:
                p_loss += prob

    return p_win, p_draw, p_loss

In [30]:
leagues = [
    "premierleague_england",
    "seriea_italy",
    "laliga_spain",
    "bundesliga_germany",
    "ligue1_france",
]

for league in leagues:
    df_future = globals()[f"fixtures_{league}"]  # corrected variable name

    attack = globals()[f"attack_{league}"]
    defense = globals()[f"defense_{league}"]
    league_avg_scored = globals()[f"league_avg_scored_{league}"]
    home_advantage = globals()[f"home_advantage_{league}"]

    results = []

    for _, row in df_future.iterrows():
        home = row["homeTeam"]
        away = row["awayTeam"]

        p_win, p_draw, p_loss = match_probabilities_league(
            home,
            away,
            attack,
            defense,
            league_avg_scored,
            home_advantage,
        )

        results.append({
            "utcDate": row["utcDate"],
            "homeTeam": home,
            "awayTeam": away,
            "p_home_win": p_win,
            "p_draw": p_draw,
            "p_away_win": p_loss,
        })

    globals()[f"df_probabilities_{league}"] = pd.DataFrame(results)

In [31]:
for league in leagues:
    print(f"\n=== {league.upper()} ===")
    print(globals()[f"df_probabilities_{league}"].head(2))


=== PREMIERLEAGUE_ENGLAND ===
                utcDate homeTeam         awayTeam  p_home_win    p_draw  p_away_win
0  2026-02-10T19:30:00Z  Chelsea     Leeds United    0.665071  0.174707    0.146950
1  2026-02-10T19:30:00Z  Everton  AFC Bournemouth    0.405617  0.262339    0.331176

=== SERIEA_ITALY ===
                utcDate  homeTeam   awayTeam  p_home_win    p_draw  p_away_win
0  2026-02-08T19:45:00Z  Juventus      Lazio    0.538112  0.247685    0.212698
1  2026-02-09T17:30:00Z  Atalanta  Cremonese    0.711450  0.185448    0.097475

=== LALIGA_SPAIN ===
                utcDate    homeTeam     awayTeam  p_home_win    p_draw  p_away_win
0  2026-02-08T20:00:00Z    Valencia  Real Madrid    0.164215  0.185291    0.640011
1  2026-02-09T20:00:00Z  Villarreal     Espanyol    0.680832  0.168425    0.135835

=== BUNDESLIGA_GERMANY ===
                utcDate           homeTeam       awayTeam  p_home_win    p_draw  p_away_win
0  2026-02-13T19:30:00Z  Borussia Dortmund       M05Mainz    0.6171

## 8. Compare calculated probabilities to bookmaker ones

In [32]:
rmse_results = {}

for league in leagues:
    print(f"\n=== {league.replace('_', ' ').title()} ===")
    
    # Load model probabilities and bookmaker odds
    df_model = globals()[f"df_probabilities_{league}"]
    df_book = globals()[f"betting_odds_{league}"]

    # Merge on home/away team names
    df_compare = df_model.merge(
        df_book,
        left_on=["homeTeam", "awayTeam"],
        right_on=["home_team", "away_team"],
        how="inner"
    )

    print("Matched rows:", len(df_compare))

    # RMSE per outcome
    rmse_home = np.sqrt(np.mean((df_compare["p_home_win"] - df_compare["p_home_book"])**2))
    rmse_draw = np.sqrt(np.mean((df_compare["p_draw"] - df_compare["p_draw_book"])**2))
    rmse_away = np.sqrt(np.mean((df_compare["p_away_win"] - df_compare["p_away_book"])**2))

    # Total RMSE across all outcomes
    rmse_total = np.sqrt(np.mean(
        (df_compare["p_home_win"] - df_compare["p_home_book"])**2 +
        (df_compare["p_draw"] - df_compare["p_draw_book"])**2 +
        (df_compare["p_away_win"] - df_compare["p_away_book"])**2
    ))

    rmse_results[league] = {
        "rmse_home": rmse_home,
        "rmse_draw": rmse_draw,
        "rmse_away": rmse_away,
        "rmse_total": rmse_total
    }

    print(f"RMSE Home: {rmse_home:.4f}, Draw: {rmse_draw:.4f}, Away: {rmse_away:.4f}")
    print(f"Total RMSE: {rmse_total:.4f}")


=== Premierleague England ===
Matched rows: 21
RMSE Home: 0.0597, Draw: 0.0399, Away: 0.0525
Total RMSE: 0.0890

=== Seriea Italy ===
Matched rows: 18
RMSE Home: 0.0542, Draw: 0.0377, Away: 0.0556
Total RMSE: 0.0863

=== Laliga Spain ===
Matched rows: 23
RMSE Home: 0.0796, Draw: 0.0578, Away: 0.0602
Total RMSE: 0.1154

=== Bundesliga Germany ===
Matched rows: 18
RMSE Home: 0.0539, Draw: 0.0524, Away: 0.0618
Total RMSE: 0.0973

=== Ligue1 France ===
Matched rows: 19
RMSE Home: 0.0746, Draw: 0.0609, Away: 0.0484
Total RMSE: 0.1078


## 9. Replace my estimates probabilities with the ones I have from odds, creating my final match probabilities

In [33]:
df_final_probabilities_all = {}

for league in leagues:
    print(f"\n=== {league.replace('_', ' ').title()} ===")
    
    df_probabilities = globals()[f"df_probabilities_{league}"]
    betting_odds_avg = globals()[f"betting_odds_{league}"]

    # Merge model and bookmaker probabilities
    df_final_probabilities = df_probabilities.merge(
        betting_odds_avg,
        left_on=["homeTeam", "awayTeam"],
        right_on=["home_team", "away_team"],
        how="left"
    )

    # Keep relevant columns
    df_final_probabilities = df_final_probabilities[[
        "utcDate",
        "homeTeam",
        "awayTeam",
        "p_home_win",
        "p_draw",
        "p_away_win",
        "p_home_book",
        "p_draw_book",
        "p_away_book",
    ]]

    # Replace model probabilities with bookmaker odds where available
    df_final_probabilities["p_home_final"] = np.where(
        df_final_probabilities["p_home_book"].notna(),
        df_final_probabilities["p_home_book"],
        df_final_probabilities["p_home_win"]
    )

    df_final_probabilities["p_draw_final"] = np.where(
        df_final_probabilities["p_draw_book"].notna(),
        df_final_probabilities["p_draw_book"],
        df_final_probabilities["p_draw"]
    )

    df_final_probabilities["p_away_final"] = np.where(
        df_final_probabilities["p_away_book"].notna(),
        df_final_probabilities["p_away_book"],
        df_final_probabilities["p_away_win"]
    )

    print("Used betting odds:", df_final_probabilities["p_home_book"].notna().sum())
    print("Used model:", df_final_probabilities["p_home_book"].isna().sum())

    # Keep only final probabilities
    df_final_probabilities = df_final_probabilities[[
        "utcDate",
        "homeTeam",
        "awayTeam",
        "p_home_final",
        "p_draw_final",
        "p_away_final"
    ]]

    df_final_probabilities_all[league] = df_final_probabilities
    display(df_final_probabilities.head(2))


=== Premierleague England ===
Used betting odds: 21
Used model: 108


,utcDate,homeTeam,awayTeam,p_home_final,p_draw_final,p_away_final
0,2026-02-10T19:30:00Z,Chelsea,Leeds United,0.599077,0.224591,0.176332
1,2026-02-10T19:30:00Z,Everton,AFC Bournemouth,0.393698,0.280429,0.325873



=== Seriea Italy ===
Used betting odds: 18
Used model: 126


,utcDate,homeTeam,awayTeam,p_home_final,p_draw_final,p_away_final
0,2026-02-08T19:45:00Z,Juventus,Lazio,0.674738,0.218121,0.107141
1,2026-02-09T17:30:00Z,Atalanta,Cremonese,0.703766,0.187490,0.108745



=== Laliga Spain ===
Used betting odds: 23
Used model: 130


,utcDate,homeTeam,awayTeam,p_home_final,p_draw_final,p_away_final
0,2026-02-08T20:00:00Z,Valencia,Real Madrid,0.174141,0.219801,0.606058
1,2026-02-09T20:00:00Z,Villarreal,Espanyol,0.562071,0.237323,0.200606



=== Bundesliga Germany ===
Used betting odds: 18
Used model: 100


,utcDate,homeTeam,awayTeam,p_home_final,p_draw_final,p_away_final
0,2026-02-13T19:30:00Z,Borussia Dortmund,M05Mainz,0.620493,0.218261,0.161247
1,2026-02-14T14:30:00Z,Werder Bremen,Bayern Munich,0.137813,0.170678,0.691508



=== Ligue1 France ===
Used betting odds: 19
Used model: 99


,utcDate,homeTeam,awayTeam,p_home_final,p_draw_final,p_away_final
0,2026-02-08T19:45:00Z,Paris Saint-Germain,Marseille,0.647281,0.192837,0.159882
1,2026-02-13T18:00:00Z,Stade Rennais,Paris Saint-Germain,0.163776,0.194880,0.641344


In [34]:
# Dictionary to store simulation dataframes for each league
df_simulation_all = {}

# Columns for probability normalization
prob_cols = ["p_home_final", "p_draw_final", "p_away_final"]

for league in leagues:
    df = df_final_probabilities_all[league].copy()
    
    # Normalize probabilities so each row sums to 1
    df[prob_cols] = df[prob_cols].div(df[prob_cols].sum(axis=1), axis=0)
    
    # Store in dictionary
    df_simulation_all[league] = df
    
    # Preview top 3 rows
    print(f"\n=== {league.replace('_', ' ').title()} ===")
    display(df.head(3))


=== Premierleague England ===


,utcDate,homeTeam,awayTeam,p_home_final,p_draw_final,p_away_final
0,2026-02-10T19:30:00Z,Chelsea,Leeds United,0.599077,0.224591,0.176332
1,2026-02-10T19:30:00Z,Everton,AFC Bournemouth,0.393698,0.280429,0.325873
2,2026-02-10T19:30:00Z,Tottenham Hotspur,Newcastle United,0.333668,0.271834,0.394498



=== Seriea Italy ===


,utcDate,homeTeam,awayTeam,p_home_final,p_draw_final,p_away_final
0,2026-02-08T19:45:00Z,Juventus,Lazio,0.674738,0.218121,0.107141
1,2026-02-09T17:30:00Z,Atalanta,Cremonese,0.703766,0.187490,0.108745
2,2026-02-09T19:45:00Z,AAS Roma,Cagliari,0.641415,0.230968,0.127617



=== Laliga Spain ===


,utcDate,homeTeam,awayTeam,p_home_final,p_draw_final,p_away_final
0,2026-02-08T20:00:00Z,Valencia,Real Madrid,0.174141,0.219801,0.606058
1,2026-02-09T20:00:00Z,Villarreal,Espanyol,0.562071,0.237323,0.200606
2,2026-02-13T20:00:00Z,Elche,Osasuna,0.335416,0.309791,0.354794



=== Bundesliga Germany ===


,utcDate,homeTeam,awayTeam,p_home_final,p_draw_final,p_away_final
0,2026-02-13T19:30:00Z,Borussia Dortmund,M05Mainz,0.620493,0.218261,0.161247
1,2026-02-14T14:30:00Z,Werder Bremen,Bayern Munich,0.137813,0.170678,0.691508
2,2026-02-14T14:30:00Z,TSG Hoffenheim,SC Freiburg,0.528208,0.248097,0.223695



=== Ligue1 France ===


,utcDate,homeTeam,awayTeam,p_home_final,p_draw_final,p_away_final
0,2026-02-08T19:45:00Z,Paris Saint-Germain,Marseille,0.647281,0.192837,0.159882
1,2026-02-13T18:00:00Z,Stade Rennais,Paris Saint-Germain,0.163776,0.194880,0.641344
2,2026-02-13T20:05:00Z,AS Monaco,Nantes,0.646717,0.200403,0.152880


## 10. Run simulations to build the Premier League table probabilities

In [40]:
def simulate_once(fixtures, table):
    """
    Simulate the remaining fixtures once and return the updated league table with positions.
    Assumes:
    - table has columns: 'team', 'pts', 'gd'
    - fixtures has columns: 'homeTeam', 'awayTeam', 'p_home_final', 'p_draw_final', 'p_away_final'
    """
    table_sim = table.copy()

    # Initialize points dict
    points = dict(zip(table_sim["team"], table_sim["pts"]))

    for _, row in fixtures.iterrows():
        home = row["homeTeam"]
        away = row["awayTeam"]

        # Outcome probabilities
        probs = [row["p_home_final"], row["p_draw_final"], row["p_away_final"]]
        outcome = np.random.choice(["H", "D", "A"], p=probs)

        if outcome == "H":
            points[home] += 3
        elif outcome == "D":
            points[home] += 1
            points[away] += 1
        else:
            points[away] += 3

    # Update table
    result_df = table_sim.copy()
    result_df["pts"] = result_df["team"].map(points)

    # Sort by points and goal difference
    result_df = result_df.sort_values(["pts", "gd"], ascending=[False, False])
    result_df["position"] = np.arange(1, len(result_df)+1)

    return result_df


def run_simulations(fixtures, table, n_sim=10000):
    """
    Run multiple simulations and return a DataFrame of position counts per team.
    """
    position_counts = {team: np.zeros(len(table)) for team in table["team"]}

    for _ in range(n_sim):
        final_table = simulate_once(fixtures, table)
        for _, row in final_table.iterrows():
            position_counts[row["team"]][row["position"]-1] += 1

    pos_df = pd.DataFrame(position_counts, index=np.arange(1, len(table)+1))
    pos_df.index.name = "position"
    return pos_df


In [ ]:
n_sim = 1000  # total simulations

position_distribution_all = {}
position_distribution_pct_all = {}

for league in leagues:
    print(f"\n=== {league.replace('_', ' ').title()} ===")
    
    fixtures = df_simulation_all[league].copy()  # simulated/future fixtures
    table = globals()[league].copy()             # league table

    # Count positions for each team
    position_counts = {team: np.zeros(len(table)) for team in table["team"]}
    
    # Run simulations
    for i in range(n_sim):
        final_table = simulate_once(fixtures, table)  # make sure simulate_once uses 'team', 'homeTeam', 'awayTeam'
        for _, row in final_table.iterrows():
            position_counts[row["team"]][row["position"]-1] += 1
        
        # Optional progress print
        if (i+1) % 100 == 0:
            print(f"{i+1}/{n_sim} simulations done...")
    
    pos_df = pd.DataFrame(position_counts, index=np.arange(1, len(table)+1))
    pos_df.index.name = "position"
    pos_df_t = pos_df.T
    pos_df_pct = pos_df_t.div(pos_df_t.sum(axis=1), axis=0) * 100
    
    position_distribution_all[league] = pos_df
    position_distribution_pct_all[league] = pos_df_pct
    
    print(f"Finished simulations for {league} ✅")

## 11. Preview and present the results graphically

In [ ]:
# Build label mapping: "position  team" (extra space for 1-9)
team_labels = (
    premierleague_england[["team", "position"]]
    .set_index("team")["position"]
    .map(lambda pos: f"{pos}{'  ' if pos < 10 else ' '}")
)

# Join position and team name into one label
team_labels = (
    premierleague_england[["team", "position"]]
    .assign(
        label=lambda df: df.apply(
            lambda r: f"{r['position']}{'&nbsp;&nbsp;&nbsp;&nbsp;' if r['position'] < 10 else '&nbsp;&nbsp;'}{r['team']}",
            axis=1
        )
    )
    .set_index("team")["label"]
)


# Apply labels to your table index
position_distribution_pct.index = position_distribution_pct.index.map(team_labels)

# Drop position column if present
position_distribution_pct = position_distribution_pct.drop(columns=["position"], errors="ignore")

# Remove index name
position_distribution_pct.index.name = None


In [ ]:
greens = plt.cm.Greens
green_cmap = LinearSegmentedColormap.from_list(
    "Greens_soft",
    greens(np.linspace(0.03, 0.65, 256))
)

vmax = 25

def zero_style(val):
    if val < 0.005:
        return "background-color: white !important;"
    return ""

# ---- transform ONLY for colouring ----
color_data = position_distribution_pct.copy()
color_data = (color_data / vmax).pow(0.65) * vmax

position_distribution_pct.style \
    .background_gradient(
        cmap=green_cmap,
        vmin=0,
        vmax=vmax,
        gmap=color_data,
        axis=None          # 🔑 THIS FIXES THE ERROR
    ) \
    .applymap(zero_style) \
    .format("{:.2f}%") \
    .set_table_styles([
        {"selector": "th", "props": [
            ("background-color", "#e6edf4"),
            ("color", "#333"),
            ("text-align", "center"),
            ("font-family", "Inter, Roboto, Arial, sans-serif"),
            ("font-size", "13px"),
            ("font-weight", "600")
        ]},

        {"selector": "th.col_heading", "props": [
            ("text-align", "center")
        ]},

        {"selector": "th.row_heading", "props": [
            ("text-align", "left"),
            ("font-size", "13px"),
            ("font-weight", "600"),
            ("white-space", "nowrap"),
            ("max-width", "250px"),
            ("overflow", "hidden"),
            ("text-overflow", "ellipsis")
        ]},

        {"selector": "tr:nth-child(odd) th.row_heading", "props": [
            ("background-color", "#fbfcfe")
        ]},
        {"selector": "tr:nth-child(even) th.row_heading", "props": [
            ("background-color", "#e6edf4")
        ]},

        {"selector": "td", "props": [
            ("text-align", "center"),
            ("font-family", "Inter, Roboto, Arial, sans-serif"),
            ("font-size", "12px"),
            ("font-weight", "500"),
            ("color", "#000")
        ]}
    ])
